In [11]:
import numpy as np

def freq_itemsets(data, sup):
    itemsize = 1
    freq_item_count = 1;
    
    k_itemset_list = []
    freq_result = {}
    all_freq_items_map = {}
    all_freq_items_list = []
    while freq_item_count > 0:
        k_itemset_list, all_freq_items_map = generate_itemsets(k_itemset_list, data, itemsize,sup,all_freq_items_map) # generate item sets for given itemsize
        if itemsize > 1:     #rules can be generated for itemsets with length > 1
            all_freq_items_list = all_freq_items_list + k_itemset_list
        freq_item_count = len(k_itemset_list)
        if freq_item_count > 0:
            freq_result.update({itemsize : freq_item_count})
        itemsize += 1
    print(freq_result)
    return freq_result, all_freq_items_list, all_freq_items_map
    
#prune infreq ones that have less support count
def prune_infreq_sets(itemset, support_threshold, freq_item_count):
    for g in list(itemset.keys()):
        if itemset.get(g) < support_threshold:
            del itemset[g]
        else :
            freq_item_count.update({g : itemset.get(g)})
    return list(itemset),freq_item_count

def generate_itemsets(prev_itemset, dataset, itemsize,support_threshold,freq_item_count):
    if itemsize == 1:
        genes_set={}
        gc = 0
        for gene_form in dataset:
            for gene in gene_form:
                gc += 1
                if gene in genes_set:
                    genes_set.update({gene:genes_set.get(gene) +1})
                else: 
                    genes_set.update({gene:1})
        freq_genes_list, freq_item_count = prune_infreq_sets(genes_set, support_threshold,freq_item_count)
        return freq_genes_list, freq_item_count
    else:
        i=0
        freq_genes_list2 = []
        for gene_first in prev_itemset[i:]:
            for gene_second in prev_itemset[i+1:]:
                if itemsize == 2:
                    # Forming a tuple with gene1 and gene2--> combinations of length 2.
                    gene_pair = []
                    gene_pair.append(gene_first)
                    gene_pair.append(gene_second)
                    count = 0
                    #Increment count if the tuple if subset of a gene form.
                    for gene_form in dataset:
                        if set(gene_pair).issubset(gene_form):
                            count += 1
                    if count >= support_threshold:
                        freq_genes_list2.append(gene_pair) 
                        freq_item_count.update({",".join(gene_pair):count})
                else:
                    match = True
                    for g in range(itemsize - 2):
                        if gene_first[g] != gene_second[g]:
                            match = False
                            break
                    if match:
                        gene_k_set = []+(gene_first)
                        gene_k_set.append(gene_second[len(gene_second) - 1])
                        sup_cnt = 0
                        #Increment count if the tuple if subset of a gene form.
                        for gene_form in dataset:
                            if set(gene_k_set).issubset(gene_form):
                                sup_cnt += 1
                        if sup_cnt >= support_threshold:
                            freq_genes_list2.append(gene_k_set)
                            freq_item_count.update({",".join(gene_k_set):sup_cnt})
                    else: 
                        break
            i += 1
    return freq_genes_list2, freq_item_count

def validate_rules(body,freq_set,confidence_percent,freq_dataset_dic):
    #confidence = sup(freq_set)/freq_count(body) since rule form followed is body ==>head.
    confidence=freq_dataset_dic.get(freq_set)/(freq_dataset_dic.get(body))
    if confidence*100 >= confidence_percent:
        return True
    else:
        return False

def generate_gene_rules(input_set_list,freq_dataset_dic,confidence_percent):
    # Rule form : body ==> head
    head_return=[]
    body_return=[]
    for input_list in input_set_list:
        head=[]
        body=[]
        for i in range(len(input_list)):
            input_copy = input_list[:]
            h=input_list[i:i+1]
            h=",".join(h)
            del input_copy[i]
            b=input_copy
            b=",".join(b)
            freq_set_str = ",".join(input_list)
            if validate_rules(b,freq_set_str,confidence_percent,freq_dataset_dic):
                if h not in head:
                    head.append(h)
                    body.append(b)
            if validate_rules(h,freq_set_str,confidence_percent,freq_dataset_dic):
                if h not in body: 
                    head.append(b)
                    body.append(h)
            if len(input_list)>2:
                for j in range(i+1, len(input_list)):
                    temp_list=[]   
                    input_copy = input_list[:]
                    temp_list.append(input_list[i])
                    temp_list.append(input_list[j])
                    input_copy.remove(input_list[i])
                    input_copy.remove(input_list[j])
                    temp_list=",".join(temp_list)
                    input_copy=",".join(input_copy)
                    if validate_rules(input_copy,freq_set_str,confidence_percent,freq_dataset_dic):
                        if temp_list not in head:
                            head.append(temp_list)
                            body.append(input_copy)
                    if validate_rules(temp_list,freq_set_str,confidence_percent,freq_dataset_dic):
                        if temp_list not in body:
                            body.append(temp_list)
                            head.append(input_copy)
                 
        head_return = head_return + head
        body_return = body_return + body
    return head_return, body_return
    

    
def template1_results(arg1, arg2, arg3, head_list, body_list):
    genes = arg3.split(",")
    print("arg1 is ",arg1," arg2 is ", arg2," arg3 is ", genes)
    rule_cnt = 0
    rule_list = []
    delete_rule = []
    if(arg1 == 'RULE'):
        for g in genes:
            if arg2 == 'ANY':
                for i in range(0,len(head_list)):
                    if (g in head_list[i]) or (g in body_list[i]):
                        r = body_list[i]+'==>'+head_list[i]
                        rule_list.append(r)
                        rule_cnt+=1
                        
            elif arg2 == 'NONE':
                for i in range(0,len(head_list)):
                    if (g in head_list[i] or (g in body_list[i])):
                        delete_rule.append(i)
                for i in range(0,len(head_list)):
                    if i not in delete_rule:
                        r = body_list[i]+'==>'+head_list[i]
                        rule_list.append(r)
                        rule_cnt+=1
                
            elif arg2 == 'NUMBER':
                for i in range(0,len(head_list)):
                    if (g in head_list[i]) or (g in body_list[i]):
                        r = body_list[i]+'==>'+head_list[i]
                        rule_list.append(r)
                        rule_cnt+=1
                    #### complete the case wrt number
    elif(arg1 == 'HEAD'):
        for g in genes:
            if arg2 == 'ANY':
                for i in range(0,len(head_list)):
                    if g in head_list[i]:
                        r = body_list[i] + "==>" + head_list[i]
                        rule_list.append(r)
                        rule_cnt+=1
            elif arg2 == 'NONE':
                for i in range(0,len(head_list)):
                    if g in head_list[i]:
                        delete_rule.append(i)
            elif arg2 == 'NUMBER':
                r = body_list[i]+'==>'+head_list[i]
                ###complete case for number.
        if arg2 == 'NONE':
            for i in range(0,len(head_list)):
                if i not in delete_rule:
                    r = body_list[i]+'==>'+head_list[i]
                    rule_list.append(r)
                    rule_cnt+=1
    elif(arg1 == 'BODY'):
        for g in genes:
            if arg2 == 'ANY':
                for i in range(0,len(body_list)):
                    if g in body_list[i]:
                        r = body_list[i] + "==>" + head_list[i]
                        rule_list.append(r)
                        rule_cnt+=1
            elif arg2 == 'NONE':
                for i in range(0,len(body_list)):
                    if g in body_list[i]:
                        delete_rule.append(i)             
            elif arg2 == 'NUMBER':
                r = body_list[i]+'==>'+head_list[i]
                ###complete case for number.
        if arg2 == 'NONE':
            for i in range(0,len(head_list)):
                if i not in delete_rule:
                    r = body_list[i]+'==>'+head_list[i]
                    rule_list.append(r)
                    rule_cnt+=1
    else:
        print("invalid input.")
    return rule_list, rule_cnt

def template2_results(arg1, arg2, head_list, body_list):
    rules_list = []
    rule_cnt = 0
    if arg1.lower() == 'rule':
        for i in range(0,len(head_list)):
            if (head_list[i].count('G') + body_list[i].count('G')) >= int(arg2):
                r = body_list[i]+'==>'+head_list[i]
                rules_list.append(r)
                rule_cnt+=1
    elif arg1.lower() == 'head':
        for i in range(0,len(head_list)):
            if (head_list[i].count('G')) >= int(arg2):
                r = body_list[i]+'==>'+head_list[i]
                rules_list.append(r)
                rule_cnt+=1
    elif arg1.lower() == 'body':
        for i in range(0,len(body_list)):
            if (body_list[i].count('G')) >= int(arg3):
                r = body_list[i]+'==>'+head_list[i]
                rules_list.append(r)
                rule_cnt+=1
    else:
        print("Invalid input.")
    return rules_list, rule_cnt

def template3_results(operation, left1,left2,left3,right1,right2,right3,head_list,body_list):
    if operation == '1or1' or operation == "1and1":
        op1_list_cnt = template1_results(left1,left2,left3,head_list,body_list)
        op2_list_cnt = template1_results(right1,right2,right3,head_list,body_list)
        if(operation =="1or1"):
            return set(op1_list_cnt[0]).union(set(op2_list_cnt[0]))
        elif(operation=="1and1"):
            return set(op1_list_cnt[0]).intersection(op2_list_cnt[0])
    elif operation == '1or2' or operation == "1and2":
        op1_list_cnt = template1_results(left1,left2,left3,head_list,body_list)
        op2_list_cnt = template2_results(right1,right2,head_list,body_list)
        if(operation =="1or2"):
            return set(op1_list_cnt[0]).union(set(op2_list_cnt[0]))
        elif(operation=="1and2"):
            return set(op1_list_cnt[0]).intersection(op2_list_cnt[0])
    elif operation == '2or2' or operation == "2and2":
        op1_list_cnt = template2_results(left1,left2,head_list,body_list)
        op2_list_cnt = template2_results(right1,right2,head_list,body_list)
        if(operation =="2or2"):
            return set(op1_list_cnt[0]).union(set(op2_list_cnt[0]))
        elif(operation=="2and2"):
            return set(op1_list_cnt[0]).intersection(op2_list_cnt[0])
    else:
        print("invalid operation.")


#Reading data and forming a list of all genes with proper names like G1_UP or G30_DOWN
#Neglecting the disease name
def transform_data(filename):
    gene_data = open(filename)
    gene_seq = gene_data.readlines()
    all_gene_list=[]
    cnt = 0;
    for line in gene_seq:
        genes = line.strip().split('\t')
        gene_set = []
        i=1
        for ele in np.array(genes):
            #if i < len(genes) :
            gene_set.append("G" + str(i) + "_" + ele)
            i = i + 1
            cnt += 1
        all_gene_list.append(gene_set)
    return all_gene_list


gene_data_list = transform_data("associationRulesData.txt")
support_percent = int(input('support percentage: '))
sup = int((support_percent/100)*len(gene_data_list))
freq_counts, itemset_list, itemset_map = freq_itemsets(gene_data_list, sup) #computing all frequent item combinations list and map with their sup counts.

#Part2B --> Rule generation and printing rules with the template given.
confidence_percent = int(input('confidence percentage: '))
head_list, body_list = generate_gene_rules(itemset_list, itemset_map, confidence_percent)
print(head_list)
print("body ***")
print(body_list)
template = input('input template type: ')

if(template == 'template1'):
    arg1 = input('rule1: RULE|HEAD|BODY')
    arg2 = input('rule2: ANY|NUMBER|NONE')
    arg3 = input('list of items , seperated.')
    rules_template1, template1_rule_cnt = template1_results(arg1, arg2, arg3, head_list, body_list)
    print("template1 cnt: ",template1_rule_cnt)
    print(rules_template1)
elif template == 'template2':
    arg1 = input('rule1: RULE|HEAD|BODY')
    arg2 = input('rule2: number')
    rules_template2, template2_rule_cnt = template2_results(arg1, arg2, head_list, body_list)
    print('template2 count: ', template2_rule_cnt)
    print(rules_template2)
elif template == 'template3':
    comb = input('input combination 1or1|1and1|1or2|1and2|2or2|2and2')
    if comb == "1or1" or comb == "1and1":
        arg1 = input('op1 rule1: RULE|HEAD|BODY')
        arg2 = input('op1 rule2: ANY|NUMBER|NONE')
        arg3 = input('op1 list of items , seperated.')
        arg4 = input('op2 rule1: RULE|HEAD|BODY')
        arg5 = input('op2 rule2: ANY|NUMBER|NONE')
        arg6 = input('op2 list of items , seperated.')
        rules_template3 = template3_results(comb,arg1, arg2,arg3,arg4,arg5,arg6, head_list, body_list)
    elif comb == "1or2" or comb == "1and2":
        arg1 = input('op1 rule1: RULE|HEAD|BODY')
        arg2 = input('op1 rule2: ANY|NUMBER|NONE')
        arg3 = input('op1 list of items , seperated.')
        arg4 = input('op2 rule1: RULE|HEAD|BODY')
        arg5 = input('op2 rule2: number')
        rules_template3 = template3_results(comb,arg1, arg2,arg3,arg4,arg5,"", head_list, body_list)
    elif comb == "2or2" or comb == "2and2":
        arg1 = input('op1 rule1: RULE|HEAD|BODY')
        arg2 = input('op1 rule2: number')
        arg4 = input('op2 rule1: RULE|HEAD|BODY')
        arg5 = input('op2 rule2: number')
        rules_template3 = template3_results(comb,arg1, arg2,"",arg4,arg5,"", head_list, body_list)
    else:
        input("Invalid combination.")
    print('template3 count: ', len(rules_template3))
    print(rules_template3)
else:
    print("Invalid template type.")

support percentage: 60
{1: 34, 2: 2}
confidence percentage: 70
['G59_Up', 'G72_Up', 'G59_Up', 'G96_Down']
body ***
['G72_Up', 'G59_Up', 'G96_Down', 'G59_Up']
input template type: template3
input combination 1or1|1and1|1or2|1and2|2or2|2and21or1
op1 rule1: RULE|HEAD|BODYHEAD
op1 rule2: ANY|NUMBER|NONEANY
op1 list of items , seperated.G59_Up,G72_Up
op2 rule1: RULE|HEAD|BODYHEAD
op2 rule2: ANY|NUMBER|NONEANY
op2 list of items , seperated.G96_Down
arg1 is  HEAD  arg2 is  ANY  arg3 is  ['G59_Up', 'G72_Up']
arg1 is  HEAD  arg2 is  ANY  arg3 is  ['G96_Down']
template3 count:  4
{'G96_Down==>G59_Up', 'G59_Up==>G96_Down', 'G59_Up==>G72_Up', 'G72_Up==>G59_Up'}
